In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import sqlite3

In [2]:
url ='https://en.wikipedia.org/wiki/List_of_largest_banks'

table_attribs = ["Rank", "Bank_name", "MC_USD_Billion"]
db_name = 'Largest_banks.db'
table_name = 'banks'
exchange_rate = 'exchange_rate.csv'
csv_path = './largest_bank.csv'


def extract(url, table_attribs):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    tables = soup.find_all('table')
    rows = tables[0].find_all('tr')
    df = pd.DataFrame(columns=table_attribs)
    for row in rows:
        cols = row.find_all('td')
        if len(cols) != 0:
            
            data = [col.get_text(strip=True) for col in cols]
            
            df.loc[len(df)] = data
    return df

In [3]:
df = extract(url, table_attribs)

In [4]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open('./etl_banks_project_log.txt', "a") as f:
        f.write(timestamp + " : " + message + "\n")
    

In [ ]:
log_progress('Data extraction complete. Initiating Transformation process')

In [7]:
def transform(df, exchange_rate):
    df["MC_USD_Billion"] = df["MC_USD_Billion"].map(
        lambda x: float(str(x).replace(',', ""))
    )
    rates = pd.read_csv(exchange_rate, index_col='Currency'). squeeze()
    df['MC_GBP_Billion'] = (df['MC_USD_Billion'] * rates['GBP']).round(2)
    df['MC_EUR_Billion'] = (df['MC_USD_Billion'] * rates['EUR']).round(2)
    df['MC_INR_Billion'] = (df['MC_USD_Billion'] * rates['INR']).round(2)
    return df

In [8]:
df = transform(df, exchange_rate)

In [9]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)


In [13]:
sql_connection = sqlite3.connect(db_name)

In [16]:
load_to_csv(df, csv_path)
load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Executing queries')

query_statement= f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT AVG(MC_GBP_Billion) from {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT Bank_name from {table_name} LIMIT 5"
run_query(query_statement, sql_connection)

SELECT * from banks
   Rank                                Bank_name  MC_USD_Billion  \
0     1  Industrial and Commercial Bank of China         6303.44   
1     2               Agricultural Bank of China         5623.12   
2     3                  China Construction Bank         5400.28   
3     4                            Bank of China         4578.28   
4     5                           JPMorgan Chase         4002.81   
..  ...                                      ...             ...   
95   96                            Handelsbanken          351.79   
96   97                 Industrial Bank of Korea          345.81   
97   98                                      DNB          339.21   
98   99                      Qatar National Bank          338.14   
99  100                  National Bank of Canada          337.65   

    MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          5042.75         5862.20       522870.35  
1          4498.50         5229.50       466437.80  
2   

In [17]:
sql_connection.close()

log_progress('Server Connection closed.')